In [1]:
import torch
print(torch.version)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

<module 'torch.version' from '/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/version.py'>
True
4
0
NVIDIA A10G


In [2]:
!git clone https://github.com/arthurcornelio88/stable-audio-tools-sam.git

fatal: destination path 'stable-audio-tools-sam' already exists and is not an empty directory.


In [3]:
!pip install stable-audio-tools

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [4]:
import os
os.environ["WANDB_API_KEY"] = "de922f7f8b9e172867a482fbc22d32acfb02042f"

In [5]:
!wandb login

wandb: Currently logged in as: arthur-cornelio (sammusic). Use `wandb login --relogin` to force relogin


In [6]:
!conda install -y -c conda-forge libsndfile

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/pytorch_p310

  added / updated specs:
    - libsndfile


The following packages will be UPDATED:

  certifi                             2024.2.2-pyhd8ed1ab_0 --> 2024.7.4-pyhd8ed1ab_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
!pip install soundfile

In [8]:
!pip install torch torchvision

In [9]:
!pip install flash-attn

In [10]:
import boto3

s3 = boto3.resource('s3')
bucket_name = 'sagemaker-eu-west-2-339713155623' #Arthur's sagemaker bucket
tar_file_key = 'webdataset_audio/audio_dataset-00001.tar'  # The key (path) to your tar archive in S3 (Arthur's)

try:
    obj = s3.Object(bucket_name, tar_file_key).get()
    print("Successfully accessed tar archive!")
except Exception as e:
    print(f"Error accessing tar archive: {e}")

Successfully accessed tar archive!


In [11]:
#télécharge le model en local
import boto3

s3 = boto3.client('s3')
s3.download_file('sagemaker-eu-west-2-339713155623', 'model.safetensors', 'model.safetensors')

import shutil

# Move the file
shutil.move('./model.safetensors', './stable-audio-tools-sam/stable_open_model_files/model.safetensors')

'./stable_open_model_files/model.safetensors'

In [12]:
#installe les bonnes version pour torch et torch audio
!pip install torch torchaudio

In [13]:
!nvidia-smi

Sat Jul 13 14:46:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1B.0 Off |                    0 |
|  0%   26C    P8              15W / 300W |      3MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
import torch
torch.cuda.set_per_process_memory_fraction(0.9)

In [15]:
import torch
torch.backends.cuda.max_split_size_mb = 128

In [16]:
!pip install -U deepspeed

In [17]:
# We need to upload manually 'dataset_config.json', model_config.json' and custom_metadataetc'
# Modify, in dataset_config, the s3_path to Arthur's

In [23]:
!pip install setuptools==65.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 70.3.0
    Uninstalling setuptools-70.3.0:
      Successfully uninstalled setuptools-70.3.0


'\nTraceback (most recent call last):\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/train.py", line 176, in <module>\n    main()\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/train.py", line 102, in main\n    model = create_model_from_config(model_config)\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/models/factory.py", line 15, in create_model_from_config\n    from .diffusion import create_diffusion_cond_from_config\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/models/diffusion.py", line 13, in <module>\n    from ..inference.generation import generate_diffusion_cond\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/inference/generation.py", line 8, in <module>\n    from .sampling import sample, sample_k, sample_rf\n  File "/home/ec2-user/SageMaker/stable-audio-tools-sam/stable_audio_tools/inference/sampling.py", line 5, in <module>\n    import k_diffusion as K\n  File "/home/ec2

In [ ]:
!python stable-audio-tools-sam/train.py --model-config stable-audio-tools-sam/stable_open_model_files/model_config.json --dataset-config stable-audio-tools-sam/stable_open_model_files/dataset_config.json --name arthur-training --save-dir checkpoints --pretrained-ckpt-path stable-audio-tools-sam/stable_open_model_files/model.safetensors --batch-size 16 --num-gpus 4 --strategy deepspeed

config.json: 100%|█████████████████████████| 1.21k/1.21k [00:00<00:00, 13.2MB/s]
spiece.model: 100%|██████████████████████████| 792k/792k [00:00<00:00, 3.55MB/s]
tokenizer.json: 100%|██████████████████████| 1.39M/1.39M [00:00<00:00, 4.66MB/s]
model.safetensors: 100%|██████████████████████| 892M/892M [00:02<00:00, 411MB/s]
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Matplotlib is building the font cache; this may take a moment.
wandb: Currently logged in as: arthur-cornelio (sammusic). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved